# 导入依赖

Cython中支持的依赖有两种,使用两种语法分别进行导入

+ python依赖,这部分和python中完全一致,因此也就不做赘述
+ cython依赖,这部分用于导入其他的cpython中`.pxd`文件中申明的内容和一些cpython封装好的标准模块. cython依赖可以在`.pxd`文件和实现文件中导入.其基本语法非常接近python的`import`,只是`import`改成了`cimport`.

+ 整体导入

```cython
cimport 模块名 [as 模块重命名]
```

+ 局部导入

```cython
from 模块名 cimport (
    接口名 [as 接口重命名],
    接口名 [as 接口重命名],
    ...
)
```

与python模块不同的地方在于,cython依赖import进来的实际是动态链接库.一个模块能被导入需要的是:

1. 有一个对应的`.pxd`文件声明接口且在寻找路径中
2. 有一个对应的动态连接库在寻找路径中或有被包装的外部库在`LD_LIBRARY_PATH`中

本文仅介绍导入部分,接口声明以及外部库的包装部分会在后面的章节中专门介绍.

## Cython库的寻找路径

接口`.pxd`文件和对应动态连接库的寻找路径规则是一样的.查找顺序为:

1. 当前模块所在目录.

2. 标准库目录,Cython提供了一些对C,C++,python以及numpy的封装作为标准库,比如`libc.math`,他们在Cython源码的[Includs](https://github.com/cython/cython/tree/master/Cython/Includes)文件夹下.

3. `sys.path`中指定的目录,和python一样,cython也使用`sys.path`指定查找目录.

4. Cython编译缓存目录,一旦一个Cython模块被编译,其结果会被缓存,Cython会查找缓存中的模块或头文件以避免重复的编译过程

需要注意的是与Python不同,Cython模块的查找路径不包括当前工作目录.因此如果你的Cython模块或头文件位于当前工作目录下你需要在`sys.path`中添加这个目录,或者使用绝对路径或相对路径来指定模块的位置.

In [4]:
%load_ext Cython

In [7]:
%%cython
cimport cython
help(cython)

NameError: name 'cython' is not defined

## Cython标准库

Cython的标准库只能说堪堪够用,主要包括:

+ `cpython`,主要是对`Python.h`的封装,包括一些基本类型和标准库,静态化以提高性能并提供cython接口.这个标准库更多的时候是给Python嵌入程序使用的,几个相对可能用到的子库包括
    + [cpython.array](https://github.com/cython/cython/blob/master/Cython/Includes/cpython/array.pxd),对应python标准库`array`
    + [cpython.time](https://github.com/cython/cython/blob/master/Cython/Includes/cpython/time.pxd)对应python的标准库`time`
    + [cpython.datetime](https://github.com/cython/cython/blob/master/Cython/Includes/cpython/datetime.pxd)对应python标准库`datetime`
    + [cpython.mem](https://github.com/cython/cython/blob/master/Cython/Includes/cpython/mem.pxd)对应在python内存管理中的操作

+ `libc`,对C标准库的封装,包括
    + [libc.complex](https://github.com/cython/cython/blob/master/Cython/Includes/libc/complex.pxd)对应`complex.h`
    + [libc.errno](https://github.com/cython/cython/blob/master/Cython/Includes/libc/errno.pxd)对应`errno.h`
    + [libc.float](https://github.com/cython/cython/blob/master/Cython/Includes/libc/float.pxd)对应`float.h`
    + [libc.limits](https://github.com/cython/cython/blob/master/Cython/Includes/libc/limits.pxd)对应`limits.h`
    + [libc.locale](https://github.com/cython/cython/blob/master/Cython/Includes/libc/locale.pxd)对应`locale.h`
    + [libc.math](https://github.com/cython/cython/blob/master/Cython/Includes/libc/math.pxd)对应`math.h`
    + [libc.setjmp](https://github.com/cython/cython/blob/master/Cython/Includes/libc/setjmp.pxd)对应`setjmp.h`
    + [libc.signal](https://github.com/cython/cython/blob/master/Cython/Includes/libc/signal.pxd)对应`signal.h`
    + [libc.stddef](https://github.com/cython/cython/blob/master/Cython/Includes/libc/stddef.pxd)对应`stddef.h`
    + [libc.stdint](https://github.com/cython/cython/blob/master/Cython/Includes/libc/stdint.pxd)对应`stdint.h`
    + [libc.stdio](https://github.com/cython/cython/blob/master/Cython/Includes/libc/stdio.pxd)对应`stdio.h`
    + [libc.stdlib](https://github.com/cython/cython/blob/master/Cython/Includes/libc/stdlib.pxd)对应`stdlib.h`
    + [libc.string](https://github.com/cython/cython/blob/master/Cython/Includes/libc/string.pxd)对应`string.h`
    + [libc.time](https://github.com/cython/cython/blob/master/Cython/Includes/libc/time.pxd)对应`time.h`
    
+ `libcpp`,对C++的stl标准库的封装,需要指定使用C++,包括
    + [libcpp.random](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/random.pxd)对应stl的`random`
    + [libcpp.algorithm](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/algorithm.pxd)对应stl的`algorithm`
    + [libcpp.atomic](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/atomic.pxd)对应stl的`atomic`
    + [libcpp.cast](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/cast.pxd)提供`dynamic_cast[T]()`,`static_cast[T]()`,`reinterpret_cast[T]()`,`const_cast[T]()`四个模版方法
    + [libcpp.limits](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/limits.pxd)对应stl的`limits`
    + [libcpp.execution](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/execution.pxd)对应stl的`execution`
    + [libcpp.functional](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/functional.pxd)对应stl的`functional`
    + [libcpp.cmath](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/cmath.pxd)对应stl的`cmath`
    + [libcpp.memory](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/memory.pxd)对应stl的`memory`
    + [libcpp.typeindex](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/typeindex.pxd)对应stl的`typeindex`,需要C++11
    + [libcpp.typeinfo](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/typeinfo.pxd)对应stl的`typeinfo`
    + [libcpp.optional](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/optional.pxd)对应stl的`optional`
    + [libcpp.utility](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/utility.pxd)对应stl的`utility`
    + [libcpp.iterator](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/iterator.pxd)对应stl的`iterator`
    + [libcpp.numbers](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/numbers.pxd)对应stl的`numbers`,需要C++20
    + [libcpp.numeric](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/numeric.pxd)对应stl的`numeric`
    + [libcpp.bit](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/bit.pxd)对应stl的`bit`,需要gcc >= 10.0或clang >= 12.0
    + [libcpp.any](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/any.pxd)对应stl的`any`
    + [libcpp.complex](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/complex.pxd)对应stl的`complex`
    + [libcpp.pair](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/pair.pxd)一个快捷方式,对应stl中`utility`的`pair`结构
    + [libcpp.vector](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/vector.pxd)对应stl的`vector`
    + [libcpp.list](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/list.pxd)对应stl的`list`
    + [libcpp.forward_list](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/forward_list.pxd)对应stl的`forward_list`
    + [libcpp.deque](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/deque.pxd)对应stl的`deque`
    + [libcpp.queue](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/queue.pxd)对应stl的`queue`
    + [libcpp.stack](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/stack.pxd)对应stl的`stack`
    + [libcpp.string](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/string.pxd)对应stl的`string`
    + [libcpp.map](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/map.pxd)对应stl的`map`
    + [libcpp.set](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/set.pxd)对应stl的`set`
    + [libcpp.unordered_map](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/unordered_map.pxd)对应stl的`unordered_map`
    + [libcpp.unordered_set](https://github.com/cython/cython/blob/master/Cython/Includes/libcpp/unordered_set.pxd)对应stl的`unordered_set`
    
+ `posix`,即POSIX标准库,C标准库的一个超集,在`Unix`和`Unix-like`操作系统上额外提供一些与系统调用相关的接口.
    + [posix.dlfcn](https://github.com/cython/cython/blob/master/Cython/Includes/posix/dlfcn.pxd)对应`dlfcn.h`,控制动态链接
    + [posix.fcntl](https://github.com/cython/cython/blob/master/Cython/Includes/posix/fcntl.pxd)对应`fcntl.h`,控制文件系统
    + [posix.ioctl](https://github.com/cython/cython/blob/master/Cython/Includes/posix/ioctl.pxd)对应`sys/ioctl.h`,控制输入输出操作
    + [posix.mman](https://github.com/cython/cython/blob/master/Cython/Includes/posix/mman.pxd)对应`sys/mman.h`,控制内存管理
    + [posix.resource](https://github.com/cython/cython/blob/master/Cython/Includes/posix/resource.pxd)对应`sys/resource.h`,控制资源操作
    + [posix.select](https://github.com/cython/cython/blob/master/Cython/Includes/posix/select.pxd)对应`sys/select.h`,提供`select`函数用于监视一组文件描述符(通常是套接字,文件句柄等)的状态,从而实现异步
    + [posix.signal](https://github.com/cython/cython/blob/master/Cython/Includes/posix/signal.pxd)对应`signal.h`,提供系统信号相关的操作
    + [posix.stat](https://github.com/cython/cython/blob/master/Cython/Includes/posix/stat.pxd)对应`sys/stat.h`和`unistd.h`,提供文件状态,文件夹操作,权限操作等功能
    + [posix.time](https://github.com/cython/cython/blob/master/Cython/Includes/posix/time.pxd)对应`sys/time.h`,提供系统时间相关的操作
    + [posix.uio](https://github.com/cython/cython/blob/master/Cython/Includes/posix/uio.pxd)对应`sys/uio.h`,提供进行I/O向量操作的功能.I/O向量操作允许你以一种高效的方式在内存和文件之间传输数据,特别是在进行大规模数据传输时非常有用
    + [posix.unistd](https://github.com/cython/cython/blob/master/Cython/Includes/posix/unistd.pxd)对应`unistd.h`,提供文件操作如`read()`,`write()`,`open()`,`close()`等功能;提供进程控制操作如`fork()`,`exec()`,`exit()`等;提供系统资源管理如`getrlimit()`,`setrlimit()`等功能;提供文件路径处理如 `chdir()`,`getcwd()`,`access()`等功能;提供进程等待如`wait()`,`waitpid()`等功能
    + [posix.wait](https://github.com/cython/cython/blob/master/Cython/Includes/posix/wait.pxd)对应`sys/wait.h`,提供进程控制功能
  
+ [openmp](https://github.com/cython/cython/blob/master/Cython/Includes/openmp.pxd),openmp封装

+ [numpy](https://github.com/cython/cython/blob/master/Cython/Includes/numpy/__init__.pxd),numpy的C接口封装,对应`numpy/arrayobject.h`,`numpy/ndarraytypes.h`,`numpy/arrayscalars.h`,`numpy/ufuncobject.h`的封装,封装了numpy的array,ufunc等常用类型
    + [numpy.math]()对应`numpy/npy_math.h`,numpy中数学计算的相关函数
    
+ `cython.operator`,cython中C/C++的操作符的函数化封装.这个库在python包cython两边是一样的.包括

    函数|对应操作符|说明
    ---|---|---
    `cython.address(foo)`|``int* ptr = &x`中对应的`&x`|用于取地址
    `cython.operator.dereference(ptr)`|`int* ptr = &x; int y = *(ptr)`中对应的`*(ptr)`|用于取消引用
    `cython.operator.preincrement(foo)`|`++(foo)`|用于预增量
    `cython.operator.predecrement(foo)`|`--(foo)`|用于预减量
    `cython.operator.postincrement(foo)`|`foo++`|用于后增量
    `cython.operator.postdecrement(foo)`|`foo--`|用于后减量
    `cython.operator.comma(foo,bar)`|`((foo),(bar))`|C++中的逗号运算符
    `cython.operator.typeid(foo)`|`typeid(foo)`| 查看C++类的信息,会返回`const libcpp.typeinfo.type_info &`